Joshua Avakian - CSCI 191T

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import numpy as np
import pandas as pd
import gc
from IPython.display import clear_output
!pip install tensorflow 2.3.0
!pip install autokeras
clear_output()
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import autokeras
%matplotlib inline

#Paths and Global Variables

In [ ]:
Train_P = "/content/train.csv"
Test_P = "/content/test.csv"
Sub_P = "/gender_submission.csv"
Sub_Path = "submission.csv"

ID = "PassengerId"
TARGET = "Survived"
TEST_SIZE = 0.2
RANDOM_SEED = 45
MAX_TRIAL = 4 # for simple test 
EPOCHS = 10  # for simple test 
VALIDATION_SPLIT = 0.15


#Check for NULL plus Train and Test functions

In [ ]:
train = pd.read_csv(Train_P)
test = pd.read_csv(Test_P)
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch']
train = train.drop(drop_elements, axis = 1)
test = test.drop(drop_elements, axis = 1)
def checkNull_fillData(df):
    for col in df.columns:
        if len(df.loc[df[col].isnull() == True]) != 0:
            if df[col].dtype == "float64" or df[col].dtype == "int64":
                df.loc[df[col].isnull() == True,col] = df[col].mean()
            else:
                df.loc[df[col].isnull() == True,col] = df[col].mode()[0]

checkNull_fillData(train)
checkNull_fillData(test)

string_list = [] 
number_list = []
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
        string_list.append(colname)
    else:
        number_list.append(colname)     
train = pd.get_dummies(train, columns=string_list)
test = pd.get_dummies(test, columns=string_list)

#Scale Data

In [ ]:
y = train[TARGET]
X = train.drop([TARGET],axis=1)
X_test = test
gc.collect()

260

#Spliting Data into training set and a validation set

In [ ]:
total = pd.concat([X,X_test],axis=0)
trans = StandardScaler()
trans.fit(total)
X= trans.transform(X)
X_test = trans.transform(X_test)

gc.collect()

99442

#Using Keras Library to not only build the best model but search for it as well

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=TEST_SIZE,random_state=RANDOM_SEED)

clf = autokeras.StructuredDataClassifier(max_trials=MAX_TRIAL,seed=RANDOM_SEED)
clf.fit(X_train, y_train, epochs=EPOCHS, validation_split=VALIDATION_SPLIT)

INFO:tensorflow:Reloading Oracle from existing project ./structured_data_classifier/oracle.json
INFO:tensorflow:Reloading Tuner from ./structured_data_classifier/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
23/23 [==============================] - 1s 2ms/step - loss: 0.6131 - accuracy: 0.6910
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4995 - accuracy: 0.7739
Epoch 3/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4715 - accuracy: 0.7626
Epoch 4/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4603 - accuracy: 0.7809
Epoch 5/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4543 - accuracy: 0.7907
Epoch 6/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.7978
Epoch 7/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4462 - accuracy: 0.8076
Epoch 8/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4427 - accuracy: 0.8090
Epoc

#Evalutation and Best Model

In [ ]:
clf.evaluate(X_val, y_val)

model = clf.export_model()
model.summary()
pred_test = model.predict(X_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.3739 - accuracy: 0.8883
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 8)                 0         
_________________________________________________________________
normalization (Normalization (None, 8)                 17        
_________________________________________________________________
dense (Dense)                (None, 128)               1152      
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
__________________________________________

#CSV file of predicted survivors

In [ ]:
sub = pd.read_csv(Sub_P)
sub[TARGET]=(pred_test > 0.5).astype(int)
sub.to_csv(Sub_Path,index=False)
sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
